<a href="https://colab.research.google.com/github/yepez26/BRFUNK_ismir_2024/blob/main/maritalk_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install maritalk

In [ ]:
import pandas as pd
import numpy as np
import maritalk
import random
import IPython
# from IPython.core.debugger import set_trace


In [ ]:


lyrics = pd.read_excel('/content/drive/MyDrive/UFRGS/mestrado/matitalk/base_estrofes_ext.xlsx')
musica_ref = pd.read_csv('/content/drive/MyDrive/UFRGS/mestrado/matitalk/base_de_ouro.csv')

In [ ]:
musica_ref

,Unnamed: 0,artista,musica,trecho,classificacao
0,67,MC Zoi de Gato,Amor Só de Mãe,para começar esse funk sem neurose eu vou dize...,Mãe
1,569,MC Lipi,Minha História,e se eu te contar a minha história nem sei co...,Mãe
2,643,MC Hariel,"Obrigado, Mãe (Pt. 2) (part. Dfideliz)",mc hariel só se fosse o lt com o murillo pra p...,Mãe
3,992,MC Neguinho do Kaxeta,"minha coroa (part. MC Cebezinho, MC Kelvinho e...",mc neguinho do kaxeta faz um tempo que eu não...,Mãe
4,416,Mc Ricardo,Hoje Eu Quero Trair a Minha Namorada/ Fui Deu,eu vou pra ousadia eu vou pra cachorrada hoje ...,Objetificação
5,660,MC Don Juan,Os Opostos Se Atraem,por que eu fui logo me apegar por você não é a...,Objetificação
6,667,MC Kevin o Chris,Papin (part. MC Caja),caiu no meu papinho já era aí o bagulho flui à...,Objetificação
7,978,MC Meno K,Vou Te Botar (part. MC Yuri),na minha casa tomou uma bala em cima da cama p...,Objetificação
8,438,MC Neguinho do Kaxeta,Inevitável/ Cheiro Bom,inevitável eu querer te evitar é mesma coisa ...,Relacionamento
9,774,MC Kevin o Chris,Relação Aberta,maloqueira cabelo enrolado sorriso apaixonante...,Relacionamento


In [ ]:

# base reduzida com músicas de teste
lyrics_test = lyrics[lyrics['musica'].isin(musica_ref['musica'])].reset_index(drop=True)


In [ ]:
#parameters

num_iter = 10 # numero de vezes que será repetida a interação
num_groups = 20 # numero de grupos diferente que será separada cada interação
num_topicos = 7 # numero de tópicos que será perguntado o modelo
aux_dic = {number: ['topics:'] for number in range(0,num_topicos)} # dicionario auxiliar para montagem de base final
tamanho = len(lyrics_test)

In [ ]:
# definindo os do prompt

model = maritalk.MariTalk(key="")
# dados os seguintes trechos de música, sugira {n} tópicos que descrevam os assuntos abordados
template1 = "dados os seguintes trechos de música, sugira {} tópicos que descrevam os assuntos abordados: ".format(num_topicos)

In [ ]:
# %xmode Verbose
# %pdb on

lista_dfs = [] # lista onde vão ser salvo os dfs de cada iteração

for iter in range(0,num_iter):

    random.seed(iter*10)
    df_1_volta = pd.DataFrame() # df com o resultado final de topicos ao longo de 1 iteração
    # definido os trechos de comparação aletóriamente usando a biblioteca random
    list_sample = []
    for i in range(0,tamanho):
      list_sample.append(random.randrange(0,num_groups))

    lyrics_test['sample'] = list_sample

    # fazendo a iteração dentro dos grupos criados
    # df_aux1 = pd.DataFrame()
    for grupo in range(0,num_groups):

        # filtrar musicas do grupo
        lista_trechos = list(lyrics_test[lyrics_test['sample']==grupo]['trecho'].reset_index(drop=True))

        # string com trechos músicas
        string_trechos = ['trecho {}: '.format(i) + lista_trechos[i] for i in range(0,len(lista_trechos))]
        string_trechos = '; '.join(string_trechos) + ' '

        #template com frases do grupo
        template2 = template1 + string_trechos

        # chamda modelo
        answer = model.generate(
            template2,
          do_sample=True,
            # max_tokens=200,
            temperature=0.5,
            top_p=0.95)

        # splitar respostas em lista
        list_answer = answer.split('\n')


        list_answer = list_answer[0:num_topicos]

        while len(list_answer) <  num_topicos:
            list_answer.append('sem_topico')


        # df auxiliar final
        df_aux = pd.DataFrame()
        df_aux['trechos'] = lista_trechos

        for col in aux_dic.keys():
          df_aux[col] = str(aux_dic[col])

        for i in range(0,num_topicos):
          # concat horizontal two list
          df_aux[i] = df_aux[i]+str(list_answer[i])

        df_1_volta = df_1_volta.append(df_aux)
        # set_trace()

    df_1_volta.to_excel('/content/drive/MyDrive/UFRGS/mestrado/matitalk/base_v2/base_topics_{}.xlsx'.format(iter))
    lista_dfs.append([df_1_volta])
    # set_trace()





<ipython-input-9-a8824bf470ad>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_volta = df_1_volta.append(df_aux)
<ipython-input-9-a8824bf470ad>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_volta = df_1_volta.append(df_aux)
<ipython-input-9-a8824bf470ad>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_volta = df_1_volta.append(df_aux)
<ipython-input-9-a8824bf470ad>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_volta = df_1_volta.append(df_aux)
<ipython-input-9-a8824bf470ad>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste